In [34]:
import cryptocompare
import pandas as pd
import datetime
import boto3
import datetime
import requests
import json
from functools import reduce
import matplotlib.pyplot as plt
import numpy as np
import re
%matplotlib inline
from datetime import datetime, timedelta
import awswrangler as wr
pd.set_option('display.max_columns', None)

import pytz

utc=pytz.UTC


CRYPTO_COMPARE_KEY = "6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4"
cryptocompare.cryptocompare._set_api_key_parameter(CRYPTO_COMPARE_KEY)

'&api_key=6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4'

In [16]:
session = boto3.Session(profile_name='master')
s3_client = session.client('s3')
bucket = 'kosmobiker-masterproject'

In [17]:
def get_list_of_objects_s3(operation_parameters, s3=None):
    """
    List files in specific S3 URL
    """
    if not s3:
        s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(**operation_parameters)
    for page in page_iterator:
        for content in page.get('Contents'):
            if content.get('LastModified') > utc.localize(datetime.now() - timedelta(days=4)):
                yield content.get('Key')


In [18]:
def get_json_s3(bucket, key, s3=None):
    """
    Function is used to get JSON file
    from S3 bucket
    """
    if not s3:
        s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8')
    return json.loads(data)

In [19]:
params = {
        'Bucket' : bucket,
        'Prefix' : "data/daily_crypto_data/"
        }

In [20]:
data = []
for key in get_list_of_objects_s3(params, s3_client):
    try:
        if key.endswith('.json'):
            coin_cur = re.findall("(?:[0-9]{4}_[0-9]{2}_[0-9]{2})_(.+)(?:.json)", key)[0]
            tmp = get_json_s3(bucket, key, s3_client)
            res = [dict(item, **{'coin_currency':f'{coin_cur}'}) for item in tmp]
            data.extend(res)
    except Exception as err:
        print(err)
#write to glue table

In [21]:
df_test = pd.DataFrame(data)

In [37]:
df_test['partition_column'] = pd.to_datetime(df_test['time'], unit='s').dt.strftime('%Y-%m-%d')
df_test

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,coin_currency,date_ticker,partition_column
0,1658968200,0.01182,0.01181,0.01181,54.31,0.6417,0.01182,direct,,BNB_BTC,2022-07-28,2022-07-28
1,1658968260,0.01182,0.01181,0.01182,17.28,0.2042,0.01182,direct,,BNB_BTC,2022-07-28,2022-07-28
2,1658968320,0.01182,0.01182,0.01182,67.30,0.7956,0.01182,direct,,BNB_BTC,2022-07-28,2022-07-28
3,1658968380,0.01182,0.01182,0.01182,80.53,0.9521,0.01182,direct,,BNB_BTC,2022-07-28,2022-07-28
4,1658968440,0.01182,0.01182,0.01182,25.72,0.3040,0.01182,direct,,BNB_BTC,2022-07-28,2022-07-28
...,...,...,...,...,...,...,...,...,...,...,...,...
224791,1659313560,0.38220,0.38210,0.38220,66547.24,25429.7300,0.38220,direct,,XRP_USD,2022-08-01,2022-08-01
224792,1659313620,0.38260,0.38210,0.38220,171853.52,65732.5400,0.38250,direct,,XRP_USD,2022-08-01,2022-08-01
224793,1659313680,0.38290,0.38250,0.38250,54896.59,21015.1600,0.38290,direct,,XRP_USD,2022-08-01,2022-08-01
224794,1659313740,0.38300,0.38280,0.38290,62084.19,23772.8900,0.38290,direct,,XRP_USD,2022-08-01,2022-08-01


In [81]:
def write_to_glue(pandas_df: pd.DataFrame, path_table, database_name, table_name, col_partition):
    try:
        wr.s3.to_parquet(
            df=pandas_df,
            index=False,
            path=path_table,
            dataset=True,
            database=database_name,
            compression='snappy',
            table=table_name,
            mode="overwrite_partitions",
            partition_cols=col_partition,
            use_threads=True,
            concurrent_partitioning=True,
            boto3_session=session
        )
    except Exception as err:
        print(err)

In [28]:
wr.catalog.databases(boto3_session=session)


,Database,Description
0,crypto_data,OHLC data + Tweets
1,default,Default Hive database
2,master_project,this is a test database


In [84]:
# write_to_glue(df, path_table=f"s3://{bucket}/data/crypto_data/", database_name="crypto_data", table_name='daily_ohlc')

In [29]:
params_tweets = {
        'Bucket' : bucket,
        'Prefix' : "data/daily_tweets/"
        }

In [30]:


def get_latest_tweets(operation_parameters, s3=None):
    """
    List files in specific S3 URL
    """
    if not s3:
        s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(**operation_parameters)
    for page in page_iterator:
        for content in page.get('Contents'):
            if content.get('LastModified') > utc.localize(datetime.now() - timedelta(days=4)):
                yield content.get('Key')

In [31]:
tweets = []
for key in get_latest_tweets(params_tweets, s3_client):
    tmp = get_json_s3(bucket, key, s3_client)
    tweets.extend(tmp)


In [32]:
df_tweets = pd.DataFrame(tweets)

In [44]:
df_tweets['partition_column'] = pd.to_datetime(df_tweets['created_at']).dt.strftime('%Y-%m-%d')
df_tweets

,id,created_at,ceil_datetime,month,weekday,day,hour,minute,QUERY,ticker,text,favorite_count,result_type,user_name,followers_count,retweet_count,partition_column
0,1552234912488468480,2022-07-27 10:10:16,2022-07-27 10:15:00,7,3,27,10,15,bitcoin,BTC,$150 12 HOURS\n\n RT &amp; Follow @freecashco...,775,popular,MrThunderCrypto,1974310,3286,2022-07-27
1,1552339362607767556,2022-07-27 17:05:19,2022-07-27 17:15:00,7,3,27,17,15,bitcoin,BTC,"#Bitcoin offers a sound ethical, technical, &a...",3401,popular,saylor,2648949,820,2022-07-27
2,1552330264239476737,2022-07-27 16:29:10,2022-07-27 16:30:00,7,3,27,16,30,bitcoin,BTC,JUST IN: $471 billion Japanese bank's subsidia...,3128,popular,BitcoinMagazine,2583451,672,2022-07-27
3,1552715449053347842,2022-07-28 17:59:45,2022-07-28 18:00:00,7,4,28,18,0,bitcoin,BTC,RT @RallyPointPR: Most everyone has heard of B...,0,recent,sazmining,1750,1,2022-07-28
4,1552715447207731200,2022-07-28 17:59:45,2022-07-28 18:00:00,7,4,28,18,0,bitcoin,BTC,RT @CaesEth: $200 1 Winner \n5x $SOL\n\nRT + ...,0,recent,mystic_eth,37,343,2022-07-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35768,1553360962299772930,2022-07-30 12:44:48,2022-07-30 12:45:00,7,6,30,12,45,Litecoin,LTC,"RT @xinvestorcom: Today, we are announcing the...",0,recent,bilitbilit,129,69,2022-07-30
35769,1553360717016956929,2022-07-30 12:43:49,2022-07-30 12:45:00,7,6,30,12,45,Litecoin,LTC,@buiminhhuy1 @litecoin_bull Look up satanic si...,3,recent,Deb14927573,605,0,2022-07-30
35770,1553360424300679168,2022-07-30 12:42:39,2022-07-30 12:45:00,7,6,30,12,45,Litecoin,LTC,[1154] #Volendam Join In at this #FREE #Liteco...,0,recent,247robby,1215,0,2022-07-30
35771,1553360366553669632,2022-07-30 12:42:26,2022-07-30 12:45:00,7,6,30,12,45,Litecoin,LTC,"RT @xinvestorcom: Today, we are announcing the...",0,recent,nim591,53,69,2022-07-30
